In [77]:
# DOLPHIN PREPROCESSING 

In [398]:
import pandas as pd
import numpy as np
import re
import json
import nltk
from word2number import w2n
from nltk.stem.snowball import SnowballStemmer
from fractions import Fraction

In [497]:
# For dolphine use:
df = pd.read_csv('trainData_univariable.csv')
df.insert(0, 'iIndex', range(1,1+len(df)))
df = df[np.invert(np.array(df['cleaned_text'].isna()))] #not sure what this is doing .. check
df = df.rename(columns={"equations":"iEquation","cleaned_text":"sQuestion","ans":"solution"})
df = df[['solution','iEquation','iIndex','text','sQuestion','unknowns']]
df.reset_index(drop=True, inplace=True)

In [498]:
char_set = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '(', ')', '+', '-', '/', '*','.','^'}
df_cleaned = pd.DataFrame()
for id_, row in df.iterrows():
    l, r = row['iEquation'].split("=", 1)
    lSet, rSet = set(l.replace(" ", "")), set(r.replace(" ", ""))
    flagl = (len(l.strip()) == 1 and not l.strip().isdigit() and len(rSet - char_set) == 0)
    flagr = (len(r.strip()) == 1 and not r.strip().isdigit() and len(lSet - char_set) == 0)
    if flagl or flagr:
        if flagr:
            df.at[id_,'iEquation'] = r + '=' + l

In [499]:
def checkEquation(s):
    if((s[0].isalpha() and (s[1]=='=' or s[2]=='=')) or (s[-1].isalpha() and (s[-2]=='=' or s[-3]=='='))):
        l, r = s.split("=", 1)
        flagl = re.search('[a-zA-Z]', l)
        flagr = re.search('[a-zA-Z]', r)
        if(flagl!=None and flagr!=None):
            return False
        return True
    else:
        return False


df["unknown_no"] = df["unknowns"].apply(lambda x : len(x)==1)
df=df[df["unknown_no"]==True]
df["checkEquation"] = df["iEquation"].apply(lambda x : checkEquation(x))
df = df[df["checkEquation"]==True]
cols = ["checkEquation","unknown_no"]
df = df.drop(cols,axis=1)
df.reset_index(drop=True, inplace=True)
print(df.shape)
# df.tail()
# df=df[:5]

(865, 6)


In [500]:
df.head()
# ids
df[df.index==41]

,solution,iEquation,iIndex,text,sQuestion,unknowns
41,10,x=25*2/5,121,"A store has 25 bikes. 2/5 of them are red. How many bikes are red?\n""/><meta property=""og:site_name"" content=""Yahoo Answers""/><meta property=""og:image"" content=""https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo.png""/><meta property=""og:image:width"" content=""500""/><meta property=""og:image:height"" content=""500""/><meta property=""og:title"" content=""Yahoo Answers""/><meta property=""og:type"" content=""website""/><meta property=""og:locale"" content=""zh_TW""/><meta property=""og:url"" content=""https://answers.yahoo.com""/><meta property=""og:description"" content=""""/><meta name=""oath:guce:consent-host"" content=""guce.yahoo.com""/><meta name=""format-detection"" content=""telephone=no""/><link rel=""apple-touch-icon-precomposed"" href=""https://s.yimg.com/tr/54d7a352d4c11dfeb7bd74c82f3fbcff.png""/><link rel=""stylesheet"" type=""text/css"" href=""https://s.yimg.com/zq/ans/frontend/vendor.fc9104bfe51e6f7bf7f6.css""/><link rel=""stylesheet"" type=""text/css"" href=""https://s.yimg.com/zq/ans/frontend/errorPage.4...",a store 25 bikes 2/5 red many bikes red,x


In [501]:
dfjson = df.to_json(orient='index')
with open('dolphin_json1.json', 'w') as f: 
    json.dump(dfjson, f)

In [502]:
with open('dolphin_json1.json') as f:
    df = json.load(f)
df = json.loads(df)

In [509]:
def split_num_and_unit(word):
    num = ''
    unit = ''
    for idx in range(len(word)):
        char = word[idx]
        if char.isdigit() or char in ['.', '/', '(', ')']:
            num += char
        else:
            unit += word[idx:]
            break
    return num, unit#.encode('utf-8')

def mask_num(seg_text_list, equ_str):
    alphas = 'abcdefghijklmnopqrstuvwxyz'
    num_list  = []
    mask_seg_text = []
    numtemp_order = []
    num_pos = []
    count = 0 
    pos=-1
    for word in seg_text_list:
        pos+=1
        if word in ['',' ']:
            continue
        try:
            word = str(w2n.word_to_num(word))
        except:
            word = word
            
        if is_number(word):
            mask_seg_text.append("temp_"+alphas[count])
            numtemp_order.append("temp_"+alphas[count])
            if '%' in word:
                mask_seg_text.append('%')
            num_list.append(word)
            num_pos.append(pos)
            count += 1
        else:
            mask_seg_text.append(word)
    mask_equ_list = []
    s_n = sorted([(w,i) for i,w in enumerate(num_list)], key=lambda x: len(str(x[0])), reverse=True)
    if '3.14%' not in equ_str and '3.1416' not in equ_str:
        equ_str = equ_str.replace('3.14', '&PI&', 15)
    for num, idx in s_n:
        if num[-1] == '%':
            num = num[:-1]
        
        equ_str = equ_str.replace(num, '&temp_'+alphas[idx]+'&', 15)
    equ_list = []
    num_set = ['0','1','2','3','4','5','6','7','8','9','%', '.']
    for elem in equ_str.split('&'):
        if 'temp' in elem or 'PI' in elem:
            equ_list.append(elem)
        else:
            start = ''
            for char in elem:
                if char not in num_set:
                    if start != '':
                        equ_list.append(start)
                    equ_list.append(char)
                    start = ''
                else:
                    start += char
            if start != '':
                equ_list.append(start)
    equ_list = [x for x in equ_list if x!=' ']
    reverse_equ_list = equ_list[::-1]
    return mask_seg_text, num_list, equ_list, reverse_equ_list,numtemp_order, num_pos
def is_number(word):
    if word[0] == '(' and word[-1] == ')':
        return True
    if '(' in word and ')' in word and '/' in word and not word[-1].isdigit():
         return True
    if word[-1] == '%' and len(word)>1:
        return True
    if word[0].isdigit():
        return True
    try:
        float(word)
        return True
    except:
        return False
    
def extract_number_and_align_per(data_per):
    wp_id = data_per['iIndex']
    seg_text = data_per['sQuestion']
    equation = data_per['iEquation']
    equation = re.sub('\[', '(', equation)
    equation = re.sub('\]', ')', equation)
    ans = data_per['solution']

    num_list = []
    word_seg_list = []
    for word in seg_text.split():
        if ((word == '') or (word == ' ')):
            continue 
        if is_number(word):
            if '(' not in word and '%' not in word and '/' not in word and ( not word[-1].isdigit() or (len(word) > 2 and not word[-2].isdigit())) :
                num, unit = split_num_and_unit(word)
                word_seg_list.append(num)
                word_seg_list.append(unit)
            elif '(' in word and word[-1] != ')' and '/' not in word and ( not word[-1].isdigit() or (len(word) > 2 and not word[-2].isdigit())):
                num, unit = split_num_and_unit(word)
                word_seg_list.append(num)
                word_seg_list.append(unit)
            else:
                word_seg_list.append(word) 
        else:
            word_seg_list.append(word)
    word_seg_list = [x for x in word_seg_list if x!=' ']
                 
    mask_seg_text_list, num_list, mask_equ_list, mask_inv_equ_list,numtemp_order, num_pos = mask_num(word_seg_list, equation)
    return [mask_seg_text_list, mask_equ_list, num_list, mask_inv_equ_list,numtemp_order, num_pos]  



In [510]:
a = ['sum', '1/9', '', '2/3', '', '5/18', 'is']
s="sum 1/9 2/3 5/18 "
s.split()

['sum', '1/9', '2/3', '5/18']

In [511]:
df_dict = {}
df_all = df
i = 0 
for key in df_all:
    elem = df_all[key]
    key = elem['iIndex']
    tuple_per = extract_number_and_align_per(elem)
    df_dict[key] = {}
    df_dict[key]['tuple'] = tuple_per
    df_dict[key]['sQuestion'] = elem['sQuestion'][:]
    df_dict[key]['iEquation'] = elem['iEquation'][:]
    df_dict[key]['solution'] = elem['solution'][:]
    df_dict[key]['iIndex'] = elem['iIndex']
len(df_dict)

865

In [512]:
def process_ans( ans,flag=False):
        try:
            float(int(ans))
            if(flag):
                print("try ",ans)
            return ans
        except:
            if '%' in ans:
                if(flag):
                    print("% hi")
                ans = ans[:-1]
                ans = float(ans)/100
                return str(ans)
            if '|' in ans:
                if(flag):
                    print("| hi")
                ans = ans.split('|')[-1]
                return str(ans)
            if (';' in ans) or ('or' in ans):
                if(flag):
                    print("; or ")
                return ans
            if ("/" in ans):
                if(flag):
                    print("// ",ans)
                return str(convert_to_float(ans))
            else:
                try:
                    if(flag):
                        print("in except")
                    s_1 = ans.find('(')
                    if s_1 >0:
                        num_pre = ans[:s_1]
                    else:
                        num_pre = 0 
    #                 print(ans)
                    s_1 = ans.find('(')
                    e_1 = ans.find(')', s_1+1)
                    num_1 = ans[s_1+1:e_1]
                    s_2 = ans.find('(', e_1)
                    e_2 = ans.find(')', s_2)
                    num_2 = ans[s_2+1:e_2]
    #                 print("process ans ",num_pre, num_1,num_2)
                    ans = float(num_pre)+float(num_1)/float(num_2)
                    return str(ans)
                except:
                    return "None"

def validate(s):
    values = s.split('/')
    return len(values) == 2 and all(i.isdigit() for i in values)
    
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split(' ')[0].split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac



def process_single_num( ans, eqn_list, index):
    alphas = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    try:
        float(ans)
        return ans, eqn_list
    except:
        if '%' in ans:
            if ans.find('/') > 0:
                ans = convert_to_float(ans.split('%')[0])
                return str(ans), eqn_list
            ans = ans[:-1]
            for itr_a in range(len(eqn_list)):
                if eqn_list[itr_a] == ans:
                    eqn_list[itr_a] = 'temp_' + alphas[index]
            if len(ans.split("%")) > 1:
                ans = str(ans.split("%")[0]) 
            ans = float(ans)/100
            for itr_a in range(len(eqn_list)):
                if eqn_list[itr_a] == str(ans):
                    eqn_list[itr_a] = 'temp_' + alphas[index]
            return str(ans), eqn_list
        else:
            s_1 = ans.find('/')
            if s_1 >0 and validate(ans):
                ans = convert_to_float(ans)
            return str(ans), eqn_list


def process_num_list( num_list, eqn_list):
    new_num_list = []
    i = 0
    for num in num_list:
        num, eqn_list =process_single_num(num, eqn_list, i)
        new_num_list.append(num)
        i += 1
    return new_num_list, eqn_list

def check_ans( num_list, template, ans):
    alphabets = 'abcdefghijklmnopqrstuvwxyz'
    new_equ = []
    for elem in template:
        if 'temp' in elem:
            new_equ.append(str(num_list[alphabets.find(elem[-1])]))
        else:
            new_equ.append(elem)
    return equ_api_1(new_equ, ans)

In [513]:
def postfix_equation(equ_list):
    try:
        equ_list = equ_list[2:]
        stack = []
        post_equ = []
        op_list = ['+', '-', '*', '/', '^']
        priori = {'^': 3, '*': 2, '/': 2, '+': 1, '-': 1}
        for elem in equ_list:
            if elem == '(':
                stack.append('(')
            elif elem == ')':
                while 1:
                    op = stack.pop()
                    if op == '(':
                        break
                    else:
                        post_equ.append(op)
            elif elem in op_list:
                while 1:
                    if not stack:
                        break
                    elif stack[-1] == '(':
                        break
                    elif priori[elem] > priori[stack[-1]]:
                        break
                    else:
                        op = stack.pop()
                        post_equ.append(op)
                stack.append(elem)
            else:
                post_equ.append(elem)
        while stack:
            post_equ.append(stack.pop())
        return post_equ
    except:
        print(equ_list)
        return None

In [514]:
i=0
for key in df_dict:
    if(key in [487,729, 971]):
        continue
    elem = df_dict[key]
    elem_tuple = elem['tuple']
    df_dict[key] = {'text':'','target_template':[],'gen_template':[],'num_list':[],'ans':0}
    df_dict[key]['text'] = ' '.join(elem_tuple[0]) # str
    df_dict[key]['numtemp_order'] = elem_tuple[4]
    df_dict[key]['num_position'] = elem_tuple[5]
    (df_dict[key]['num_list'], df_dict[key]['target_template']) = process_num_list(elem_tuple[2], elem_tuple[1]) # [float, float]
    if(key==119):
        print(elem['solution'])
        print(process_ans(elem['solution'],False))
    df_dict[key]['post_template'] = postfix_equation(df_dict[key]['target_template'])
    df_dict[key]['ans'] = process_ans(elem['solution'],False) # float 
len(df_dict)

13/38
0.34210526315789475


865

In [515]:
df_dict_new = {}
ids=[]
tokens_allowed = ['+', '*', '-', '/', '1','100', 'PI', 'temp_m', 'temp_l', 'temp_o', 'temp_n', 'temp_i', 'temp_h', 'temp_k', 'temp_j', 'temp_e', 'temp_d', 'temp_g', 'temp_f', 'temp_a', 'temp_c', 'temp_b', '^']
for el in df_dict:
    if(el in [487,729, 971]):
        continue 
    temp_list = df_dict[el]["target_template"]
    temp_list=temp_list[2:]
    
    flag=0
    for sym in temp_list:
        if(sym not in tokens_allowed):
            flag=1
            ids.append(el)
            break
    if(flag==1):
        continue
    else:
        df_dict_new[el] = df_dict[el]
len(df_dict_new)

204

In [516]:
len(df_dict_new),len(ids)

(204, 659)

In [525]:
dff = pd.DataFrame.from_dict(df_dict_new)
dff = dff.T

def checkFormat(x):
    if(x[-1].isalpha() and x[-2]=='='):
        return False
    return True
dff["check"] = dff["target_template"].apply(lambda x: checkFormat(x))
dff[dff["check"]==False].head()
dff = dff[dff["check"]==True]
dff[dff["ans"]=="None"].shape
dff = dff[dff["ans"]!="None"]
dff = dff.drop(["check"],axis=1)
dff=dff.reset_index(drop=True)
dff.shape

(192, 8)

In [526]:
datajson = dff.to_json(orient='index')
with open("dolphin_processed.json", 'w') as f:
    json.dump(datajson, f)

In [528]:
dff.head()

,text,target_template,gen_template,num_list,ans,numtemp_order,num_position,post_template
0,sum temp_a temp_b temp_c is,"[x, =, temp_a, +, temp_b, +, temp_c]",[],"[0.1111111111111111, 0.6666666666666666, 0.2777777777777778]",1.056,"[temp_a, temp_b, temp_c]","[1, 2, 3]","[temp_a, temp_b, +, temp_c, +]"
1,number temp_a % temp_b,"[n, =, temp_a, /, 100, *, temp_b]",[],"[0.17, 200]",34,"[temp_a, temp_b]","[1, 2]","[temp_a, 100, /, temp_b, *]"
2,temp_a temp_b,"[x, =, temp_a, *, temp_b]",[],"[0.3333333333333333, 0.25]",1.0,"[temp_a, temp_b]","[0, 1]","[temp_a, temp_b, *]"
3,temp_a hens together lay a total temp_b eggs a temp_c day period many eggs would temp_d hens lay a temp_e day period,"[t, =, temp_d, *, 1, /, temp_a, *, temp_d]",[],"[5, 5, 5, 500, 500]",50000,"[temp_a, temp_b, temp_c, temp_d, temp_e]","[0, 6, 9, 15, 19]","[temp_d, 1, *, temp_a, /, temp_d, *]"
4,find area a playground temp_a long temp_b wide,"[x, =, temp_a, *, temp_b]",[],"[20, 15]",300,"[temp_a, temp_b]","[4, 6]","[temp_a, temp_b, *]"


In [307]:
datajson = dff.to_json(orient='index')
with open("dolphin_processed.json", 'w') as f:
    json.dump(datajson, f)